In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split


# Créer un écrivain SummaryWriter
writer = SummaryWriter('runs/melanoma_experiment_1')  # Ajustez le chemin

# Vérifier si CUDA est disponible, sinon utiliser le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Architecture simplifiée du CNN
class MelanomaCNN(nn.Module):
    def __init__(self):
        super(MelanomaCNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.fc = nn.Linear(16 * 112 * 112, 1)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16 * 112 * 112)
        x = torch.sigmoid(self.fc(x))
        return x


# Ensemble de données personnalisé
class MelanomaDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, limit=None):
        self.labels_df = pd.read_csv(csv_file).head(limit)  # Limite à 100 images
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0] + ".jpg")
        image = Image.open(img_name)
        label = torch.tensor(self.labels_df.iloc[idx, 1], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

In [2]:
# Transformations pour les images
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Charger l'ensemble de données
full_dataset = MelanomaDataset(csv_file='C:\\Users\\benoi\\Downloads\\isic-2020-resized\\train-labels.csv',  # Ajustez le chemin
                                img_dir='C:\\Users\\benoi\\Downloads\\isic-2020-resized\\train-resized\\train-resized',     # Ajustez le chemin
                                transform=transform)

# Séparer l'ensemble de données en ensembles d'entraînement et de validation
train_size = int(0.8 * len(full_dataset))  # 80% pour l'entraînement
val_size = len(full_dataset) - train_size  # 20% pour la validation
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, pin_memory=True)

# Charger ResNet50 pré-entraîné
model = models.resnet50(pretrained=True)
model = model.to(device)  

# Décongeler les dernières couches
# for name, param in model.named_parameters():
#     if "layer4" in name or "fc" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

# Remplacer la dernière couche fc pour notre classification binaire
model.fc = nn.Sequential(
    nn.Linear(2048, 1),
    nn.Sigmoid()
)
model.fc = model.fc.to(device) 

# Fonction de perte et optimiseur
weights = [0.1, 0.9]  # Poids pour chaque classe
class_weights = torch.FloatTensor(weights).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Scheduler pour le taux d'apprentissage
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Entraîner le modèle
epochs = 15  # Augmenter le nombre d'époques
for epoch in range(epochs):
    # Entraînement
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        outputs = model(data).squeeze()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")
    writer.add_scalar('Training Loss', total_loss / len(train_loader), epoch)

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data).squeeze()
            loss = criterion(outputs, target)
            total_val_loss += loss.item()
    print(f"Validation Loss: {total_val_loss / len(val_loader)}")
    writer.add_scalar('Validation Loss', total_val_loss / len(val_loader), epoch)

    # Mise à jour du taux d'apprentissage
    scheduler.step()

writer.close()

C:\Users\benoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\benoi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Sauvegarder le modèle
torch.save(model.state_dict(), 'melanoma_model.pth')  # Ajustez le chemin
